In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.stats as stats
import operator
import math
import warnings
import openpyxl
import random
warnings.filterwarnings('ignore')
from sklearn import tree
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, accuracy_score
import tensorflow as tf
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import ClusterCentroids
from imblearn.under_sampling import CondensedNearestNeighbour
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
import pandas as pd

# 한글 글꼴체 변경
plt.rcParams['font.family'] ='Malgun Gothic'
# df.head() 이런거 했을 때, 컬럼이 생략되지 않고, 모든 컬럼 뜨게
pd.set_option('display.max_columns', None)
# 지수 표기법 대신에 소수점으로 표시하는코드
pd.options.display.float_format = '{:.5f}'.format
# 값 길이 제한 X
pd.set_option('display.max_colwidth', None) # 값 길이 제한 없음

### ✅ Customer Segmentation

모델이 예측한 해지율을 기반으로 고객을 "양호", "보통", "위험" 3가지 그룹으로 세분화

🤔 어떻게 세분화?

#### 모델이 예측한 해지율을 뽑아보자.

In [2]:
import joblib
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder

# 모델 불러오기
model_path = 'file_pkl/lightgbm_model.pkl'
loaded_model = joblib.load(model_path)
# 스케일러 불러오기
robust_scaler = joblib.load('file_pkl/robust_scaler.pkl')
minmax_scaler = joblib.load('file_pkl/minmax_scaler.pkl')

# 데이터 불러오기
df_real = pd.read_csv('data/TPS_cancel_data_Final.csv')
df = df_real

df_modeling = df.drop(columns=['sha2_hash','p_mt','churn'])

# 레이블 인코딩 수행
label_encoders = {}
for column in df_modeling.select_dtypes(include=['object']).columns:  # object 타입 컬럼만 선택
    le = LabelEncoder()
    df_modeling[column] = le.fit_transform(df_modeling[column])  # 레이블 인코딩 수행
    label_encoders[column] = le  # 각 컬럼의 LabelEncoder 저장
# ---------------------------------------------------------------------------------------------

# 스케일링 수행
robust_columns = ['TOTAL_USED_DAYS', 'CH_HH_AVG_MONTH1']
minmax_columns = [col for col in df_modeling.columns if col not in robust_columns + ['churn']]
# 저장된 스케일러로 스케일링 수행
df_modeling[robust_columns] = robust_scaler.transform(df_modeling[robust_columns])  # RobustScaler 적용
df_modeling[minmax_columns] = minmax_scaler.transform(df_modeling[minmax_columns])  # MinMaxScaler 적용

probabilities = loaded_model.predict_proba(df_modeling)  # 각 클래스에 대한 확률 반환
predictions = (probabilities[:, 1] >= 0.5).astype(int)  # 기본 Threshold = 0.5 사용, 기본으로 Threshold는 0.5로 적용됩니다.

# 4. 원래 데이터에 예측 결과 추가
df['probability_0'] = probabilities[:, 0]  # 해지하지 않을 확률
df['probability_1'] = probabilities[:, 1]  # 해지할 확률 -> 이거 위주로 보셔야합니다
df['prediction'] = predictions             # 예측 결과 (0: 미해지, 1: 해지)

In [3]:
df

,sha2_hash,p_mt,SCRB_PATH_NM_GRP,INHOME_RATE,TOTAL_USED_DAYS,CH_LAST_DAYS_BF_GRP,STB_RES_1M_YN,AGMT_KIND_NM,BUNDLE_YN,TV_I_CNT,AGMT_END_SEG,AGE_GRP10,VOC_STOP_CANCEL_MONTH1_YN,CH_HH_AVG_MONTH1,MONTHS_REMAINING,PROD_NM_GRP,MEDIA_NM_GRP,VOC_TOTAL_MONTH1_YN,churn,probability_0,probability_1,prediction
0,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,2,I/B,0.00000,733,3개월내없음,N,신규,Y,3.00000,약정만료전 12개월이상,60대,N,0.00000,13,이코노미,HD,N,N,0.37332,0.62668,1
1,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,3,I/B,0.00000,764,일주일내,Y,신규,Y,3.00000,약정만료전 9~12개월,60대,N,6.72000,12,이코노미,HD,Y,N,0.39693,0.60307,1
2,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,4,I/B,10.00000,794,3주일전,N,신규,Y,3.00000,약정만료전 9~12개월,60대,N,9.86000,11,이코노미,HD,N,N,0.48195,0.51805,1
3,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,5,I/B,10.00000,825,4주일전,N,신규,Y,3.00000,약정만료전 9~12개월,60대,N,5.95000,10,이코노미,HD,N,N,0.53341,0.46659,0
4,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,6,I/B,0.00000,855,일주일내,N,신규,Y,3.00000,약정만료전 6~9개월,60대,N,4.03000,9,이코노미,HD,N,N,0.66975,0.33025,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22099079,fffffa7eda8144ce27e65690933ae8994e6962fefd24c982990467add99d61a7,8,현장경로,10.00000,2338,일주일내,N,신규,N,1.00000,약정만료후 12개월이상,50대,N,2.03000,-40,베이직,HD,N,N,0.61504,0.38496,0
22099080,fffffa7eda8144ce27e65690933ae8994e6962fefd24c982990467add99d61a7,9,현장경로,10.00000,2368,일주일내,N,신규,N,1.00000,약정만료후 12개월이상,50대,N,2.91000,-41,베이직,HD,N,N,0.62316,0.37684,0
22099081,fffffa7eda8144ce27e65690933ae8994e6962fefd24c982990467add99d61a7,10,현장경로,10.00000,2399,2주일전,N,신규,N,1.00000,약정만료후 12개월이상,50대,N,1.97000,-42,베이직,HD,Y,N,0.13084,0.86916,1
22099082,fffffa7eda8144ce27e65690933ae8994e6962fefd24c982990467add99d61a7,11,현장경로,0.00000,2429,3개월내없음,N,신규,N,1.00000,약정만료후 12개월이상,50대,N,0.00000,-43,베이직,HD,N,N,0.33868,0.66132,1


probability_1 = 모델이 생각한 해지율 <br><br>
prediction = 모델이 생각한 해지 여부 (0.5 (threshold) 이상이면 해지라고 예측)

In [4]:
# 일단 유지확률은 지워놓자

df.drop(columns=['probability_0'], axis=1, inplace=True)

In [5]:
df

,sha2_hash,p_mt,SCRB_PATH_NM_GRP,INHOME_RATE,TOTAL_USED_DAYS,CH_LAST_DAYS_BF_GRP,STB_RES_1M_YN,AGMT_KIND_NM,BUNDLE_YN,TV_I_CNT,AGMT_END_SEG,AGE_GRP10,VOC_STOP_CANCEL_MONTH1_YN,CH_HH_AVG_MONTH1,MONTHS_REMAINING,PROD_NM_GRP,MEDIA_NM_GRP,VOC_TOTAL_MONTH1_YN,churn,probability_1,prediction
0,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,2,I/B,0.00000,733,3개월내없음,N,신규,Y,3.00000,약정만료전 12개월이상,60대,N,0.00000,13,이코노미,HD,N,N,0.62668,1
1,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,3,I/B,0.00000,764,일주일내,Y,신규,Y,3.00000,약정만료전 9~12개월,60대,N,6.72000,12,이코노미,HD,Y,N,0.60307,1
2,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,4,I/B,10.00000,794,3주일전,N,신규,Y,3.00000,약정만료전 9~12개월,60대,N,9.86000,11,이코노미,HD,N,N,0.51805,1
3,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,5,I/B,10.00000,825,4주일전,N,신규,Y,3.00000,약정만료전 9~12개월,60대,N,5.95000,10,이코노미,HD,N,N,0.46659,0
4,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,6,I/B,0.00000,855,일주일내,N,신규,Y,3.00000,약정만료전 6~9개월,60대,N,4.03000,9,이코노미,HD,N,N,0.33025,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22099079,fffffa7eda8144ce27e65690933ae8994e6962fefd24c982990467add99d61a7,8,현장경로,10.00000,2338,일주일내,N,신규,N,1.00000,약정만료후 12개월이상,50대,N,2.03000,-40,베이직,HD,N,N,0.38496,0
22099080,fffffa7eda8144ce27e65690933ae8994e6962fefd24c982990467add99d61a7,9,현장경로,10.00000,2368,일주일내,N,신규,N,1.00000,약정만료후 12개월이상,50대,N,2.91000,-41,베이직,HD,N,N,0.37684,0
22099081,fffffa7eda8144ce27e65690933ae8994e6962fefd24c982990467add99d61a7,10,현장경로,10.00000,2399,2주일전,N,신규,N,1.00000,약정만료후 12개월이상,50대,N,1.97000,-42,베이직,HD,Y,N,0.86916,1
22099082,fffffa7eda8144ce27e65690933ae8994e6962fefd24c982990467add99d61a7,11,현장경로,0.00000,2429,3개월내없음,N,신규,N,1.00000,약정만료후 12개월이상,50대,N,0.00000,-43,베이직,HD,N,N,0.66132,1


### 모델이 1(해지)로 예측했을 때, 실제 해지인 사람들 위주로 확인해보자.
실제 해지인데, 모델이 1로 예측할만큼 가장 위험한 사람들이다.

In [34]:
check_df = df[(df['churn'] == 'Y') & (df['prediction'] == 1)]

In [35]:
check_df['probability_1'].describe()

count   97400.00000
mean        0.82435
std         0.15662
min         0.50000
25%         0.69029
50%         0.87108
75%         0.97080
max         0.99843
Name: probability_1, dtype: float64

월 별로 확인해보자

In [36]:
# 2월
print("2월")
display(check_df[(check_df['p_mt'] == 2)]['probability_1'].describe())
print("--------------------------------------")

# 3월
print("3월")
display(check_df[(check_df['p_mt'] == 3)]['probability_1'].describe())
print("--------------------------------------")

# 4월
print("4월")
display(check_df[(check_df['p_mt'] == 4)]['probability_1'].describe())
print("--------------------------------------")

# 5월
print("5월")
display(check_df[(check_df['p_mt'] == 5)]['probability_1'].describe())
print("--------------------------------------")

# 6월
print("6월")
display(check_df[(check_df['p_mt'] == 6)]['probability_1'].describe())
print("--------------------------------------")

# 7월
print("7월")
display(check_df[(check_df['p_mt'] == 7)]['probability_1'].describe())
print("--------------------------------------")

# 8월
print("8월")
display(check_df[(check_df['p_mt'] == 8)]['probability_1'].describe())
print("--------------------------------------")

# 9월
print("9월")
display(check_df[(check_df['p_mt'] == 9)]['probability_1'].describe())
print("--------------------------------------")

# 10월
print("10월")
display(check_df[(check_df['p_mt'] == 10)]['probability_1'].describe())
print("--------------------------------------")

# 11월
print("11월")
display(check_df[(check_df['p_mt'] == 11)]['probability_1'].describe())
print("--------------------------------------")

2월


count   9839.00000
mean       0.80939
std        0.15813
min        0.50003
25%        0.67206
50%        0.84120
75%        0.96158
max        0.99814
Name: probability_1, dtype: float64

--------------------------------------
3월


count   9289.00000
mean       0.82057
std        0.15782
min        0.50014
25%        0.68273
50%        0.86431
75%        0.96845
max        0.99827
Name: probability_1, dtype: float64

--------------------------------------
4월


...

--------------------------------------
10월


count   10644.00000
mean        0.82907
std         0.15602
min         0.50003
25%         0.69496
50%         0.88102
75%         0.97315
max         0.99843
Name: probability_1, dtype: float64

--------------------------------------
11월


count   9951.00000
mean       0.81497
std        0.15569
min        0.50039
25%        0.67853
50%        0.85113
75%        0.96439
max        0.99827
Name: probability_1, dtype: float64

--------------------------------------


check_df의 2월 평균 해지율 : 0.80939<br><br>
check_df의 3월 평균 해지율 : 0.82057<br><br>
check_df의 4월 평균 해지율 : 0.81696<br><br>
check_df의 5월 평균 해지율 : 0.83046<br><br>
check_df의 6월 평균 해지율 : 0.82805<br><br>
check_df의 7월 평균 해지율 : 0.82655<br><br>
check_df의 8월 평균 해지율 : 0.83667<br><br>
check_df의 9월 평균 해지율 : 0.83039<br><br>
check_df의 10월 평균 해지율 : 0.82907<br><br>
check_df의 11월 평균 해지율 : 0.81497<br><br>

실제 해지인데, 모델이 1로 예측할만큼 가장 위험한 사람들의 평균 해지율은 0.8 이상이다. <br><br>
그러므로, **0.8 이상**은 "매우 위험"에 속하는 고객들로 간주한다.

---

### 모델이 0(유지)으로 예측했을 때, 실제 유지인 사람들 위주로 확인해보자.
실제 유지인데, 모델이 0으로 예측한만큼 가장 안전한 사람들이다.

In [37]:
check_df2 = df[(df['churn'] == 'N') & (df['prediction'] == 0)]

In [39]:
check_df2['probability_1'].describe()

count   17849319.00000
mean           0.24369
std            0.11600
min            0.02017
25%            0.14994
50%            0.22680
75%            0.32836
max            0.50000
Name: probability_1, dtype: float64

월 별로 확인

In [41]:
# 2월
print("2월")
display(check_df2[(check_df2['p_mt'] == 2)]['probability_1'].describe())
print("--------------------------------------")

# 3월
print("3월")
display(check_df2[(check_df2['p_mt'] == 3)]['probability_1'].describe())
print("--------------------------------------")

# 4월
print("4월")
display(check_df2[(check_df2['p_mt'] == 4)]['probability_1'].describe())
print("--------------------------------------")

# 5월
print("5월")
display(check_df2[(check_df2['p_mt'] == 5)]['probability_1'].describe())
print("--------------------------------------")

# 6월
print("6월")
display(check_df2[(check_df2['p_mt'] == 6)]['probability_1'].describe())
print("--------------------------------------")

# 7월
print("7월")
display(check_df2[(check_df2['p_mt'] == 7)]['probability_1'].describe())
print("--------------------------------------")

# 8월
print("8월")
display(check_df2[(check_df2['p_mt'] == 8)]['probability_1'].describe())
print("--------------------------------------")

# 9월
print("9월")
display(check_df2[(check_df2['p_mt'] == 9)]['probability_1'].describe())
print("--------------------------------------")

# 10월
print("10월")
display(check_df2[(check_df2['p_mt'] == 10)]['probability_1'].describe())
print("--------------------------------------")

# 11월
print("11월")
display(check_df2[(check_df2['p_mt'] == 11)]['probability_1'].describe())
print("--------------------------------------")

2월


count   1611861.00000
mean          0.24420
std           0.11626
min           0.02035
25%           0.15007
50%           0.22665
75%           0.32903
max           0.50000
Name: probability_1, dtype: float64

--------------------------------------
3월


count   1607521.00000
mean          0.24903
std           0.11644
min           0.02215
25%           0.15428
50%           0.23448
75%           0.33561
max           0.50000
Name: probability_1, dtype: float64

--------------------------------------
4월


...

--------------------------------------
10월


count   1624585.00000
mean          0.24233
std           0.11586
min           0.02300
25%           0.14863
50%           0.22563
75%           0.32665
max           0.50000
Name: probability_1, dtype: float64

--------------------------------------
11월


count   1608452.00000
mean          0.24526
std           0.11581
min           0.02187
25%           0.15295
50%           0.22706
75%           0.32972
max           0.50000
Name: probability_1, dtype: float64

--------------------------------------


check_df의 2월 평균 해지율 : 0.24420<br><br>
check_df의 3월 평균 해지율 : 0.24903<br><br>
check_df의 4월 평균 해지율 : 0.24355<br><br>
check_df의 5월 평균 해지율 : 0.24426<br><br>
check_df의 6월 평균 해지율 : 0.24462<br><br>
check_df의 7월 평균 해지율 : 0.24450<br><br>
check_df의 8월 평균 해지율 : 0.24328<br><br>
check_df의 9월 평균 해지율 : 0.23844<br><br>
check_df의 10월 평균 해지율 : 0.24233<br><br>
check_df의 11월 평균 해지율 : 0.24526<br><br>

실제 유지인데, 모델이 0으로 예측할만큼 가장 위험한 사람들의 평균 해지율은 0.25 이하이다. <br><br>
그러므로, **0.25 이하**는 "안정"에 속하는 고객들로 간주한다.

가운데 구간을 어떻게 나눌지 몰라서, <br><br>
일단 가운데 구간을 3개의 구간으로 나눈다.<br><br>
1. 0.8 이상 (매우 위험)
2. **0.8 미만 0.6 이상**
3. **0.6 미만 0.4 이상**
4. **0.4 미만 0.25 초과**
5. 0.25 이하 (안정)

In [42]:
# 구간 나누기
def classify_customer_fine(probability):
    if probability >= 0.8:
        return '매우 위험'
    elif probability >= 0.6:
        return '위험'
    elif probability >= 0.4:
        return '주의'
    elif probability > 0.25:
        return '양호'
    else:
        return '안정'
# 분류 적용
df['customer_segmentation'] = df['probability_1'].apply(classify_customer_fine)

In [49]:
# 결과 확인
display(df['customer_segmentation'].value_counts())

customer_segmentation
안정       10113413
양호        5463499
주의        3945544
위험        1859425
매우 위험      717203
Name: count, dtype: int64

**안정 > 양호 > 주의 > 위험 > 매우 위험** 순으로 고객이 배치되어있다. <br><br>
위험도가 낮을수록 고객의 수가 점진적으로 낮아짐. 자연스럽게 계층적으로 구분이 되어 있음.
- 안정 : 약 45%
- 양호 : 약 24%
- 주의 : 약 17%
- 위험 : 약 8%
- 매우 위험 : 약 3%

### ⭐ 이제, 각 Segmentation별 실제 해지율을 확인해보자.

In [55]:
# 그룹별 평균 확률과 해지율 계산
# df['churn'] = df['churn'].map({'Y': 1, 'N': 0})

group_stats = df.groupby('customer_segmentation').agg({
    # 해당 그룹의 실제 해지율을 보여줍니다.
    'churn': 'mean'
})

In [57]:
group_stats = group_stats.sort_values(by='churn', ascending=False)
print("")
display(group_stats)

,churn
customer_segmentation,
매우 위험,0.08172
위험,0.01418
주의,0.00602
양호,0.00287
안정,0.00108


위험도가 높은 그룹일수록 해지율이 증가하는 완벽한 계층적 구조를 보임. <br><br>
특정 그룹에 데이터가 과도하게 몰려있지 않고, **"안정 → 양호 → 주의 → 위험 → 매우 위험"** 순으로 자연스럽게 계층화 되어있음. <br><br>
"매우 위험" 고객의 해지율이 8.17%, "위험" 고객이 1.42%, "안정" 고객이 0.11%로, **위험도가 높을수록 해지율이 높게 나타남.** <br><br>
"매우 위험" 고객은 "안정" 고객 대비 해지율이 **약 75배 높음**!!<br><br>
이는 모델이 해지 가능성이 높은 고객을 "위험군"으로 정확히 분류하고 있다는 증거임.

### ✅ 결론 : 고객의 구간을 총 5개의 구간으로 나누고, 매우 위험, 위험, 주의 구간이 위험구간으로 파악됨.